<a href="https://colab.research.google.com/github/aaaksenova/NLP_ABSA_project/blob/change/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# Отмечали в этом ресторане день рождение
# O        O O    B-Whole   O    O

In [3]:
!pip install stanza

     |████████████████████████████████| 432 kB 4.2 MB/s 
     |████████████████████████████████| 170 kB 53.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=554b558a6142fcdd02768619f0744bb880069e32c2d85cc3d085b154ba08b609
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [4]:
import stanza

In [5]:
stanza.download('ru')

2021-12-23 06:53:04 INFO: Downloading default packages for language: ru (Russian)...


2021-12-23 06:53:29 INFO: Finished downloading models and saved to /root/stanza_resources.


In [6]:
nlp = stanza.Pipeline('ru', processors='tokenize,pos')

2021-12-23 06:53:29 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-12-23 06:53:29 INFO: Use device: cpu
2021-12-23 06:53:29 INFO: Loading: tokenize
2021-12-23 06:53:29 INFO: Loading: pos
2021-12-23 06:53:29 INFO: Done loading processors!


In [7]:
train_asp = pd.read_csv(
    'train_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_reviews.txt', delimiter='\t', names=['text_id','text'])

In [8]:
# process text https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2
nlp(train_texts['text'][0])

[
  [
    {
      "id": 1,
      "text": "День",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
      "start_char": 0,
      "end_char": 4
    },
    {
      "id": 2,
      "text": "8-го",
      "upos": "ADJ",
      "feats": "Case=Gen|Degree=Pos|Gender=Masc|Number=Sing",
      "start_char": 5,
      "end_char": 9
    },
    {
      "id": 3,
      "text": "марта",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing",
      "start_char": 10,
      "end_char": 15
    },
    {
      "id": 4,
      "text": "прошёл",
      "upos": "VERB",
      "feats": "Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act",
      "start_char": 16,
      "end_char": 22
    },
    {
      "id": 5,
      "text": ",",
      "upos": "PUNCT",
      "start_char": 22,
      "end_char": 23
    },
    {
      "id": 6,
      "text": "можно",
      "upos": "ADV",
      "feats": "Degree=Pos",
      "start_char": 24,
      "e

In [9]:
reviews = {}
with open('train_reviews.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    reviews[line[0]] = line[1]

In [10]:
from collections import defaultdict
aspects = defaultdict(list)

In [11]:
with open('train_aspects.txt') as f:
  for line in f:
    line = line.rstrip('\r\n').split('\t')
    keys = ('category', 'mention', 'start', 'end', 'sentiment')
    # ['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
    # тут можно отдельно запомнить начало и конец каждого упоминания
    aspects[line[0]].append(dict(zip(keys, line[1:])))

In [12]:
from tqdm.auto import tqdm

In [34]:
# здесь код для упоминаний из 1 токена
# ВАЖНО: для более длинных упоминаний нужно доделать ещё немного
texts = []
bios = []
poses = []
sentences = []
sent_id=0
for text_id, text in tqdm(reviews.items()):
  processed = nlp(text)
  text_data = []
  bio_data = []
  pos_data = []
  count_token = 0
  for token in processed.iter_tokens():
    add = False
    for mention in aspects[text_id]:
      if token.start_char == int(mention['start']) and token.end_char == int(mention['end']):
        text_data.append(token.text)
        bio_data.append('B-'+mention['category'])
        pos_data.append(token.words[0].pos)
        count_token += 1
        add = True
    if not add:
      text_data.append(token.text)
      bio_data.append('O')
      pos_data.append(token.words[0].pos)
      count_token += 1
  texts.extend(text_data)
  bios.extend(bio_data)
  poses.extend(pos_data)
  sentences.extend([f'Sentence: {sent_id}'] + ['']*(count_token - 1))
  sent_id+=1

  0%|          | 0/284 [00:00<?, ?it/s]

In [33]:
len(sentences)

284

In [25]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [35]:
df = pd.DataFrame({'Sentence #' : sentences, 'Word' : texts, 'POS' : poses, 'Tag' : bios})

In [27]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((31648, 8039), (31648,))

In [89]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


-- Epoch 1
-- Epoch 1
Norm: 24.66, NNZs: 321, Bias: -2.000000, T: 31648, Avg. loss: 0.011944
Total training time: 0.52 seconds.
-- Epoch 1
Norm: 38.13, NNZs: 773, Bias: -3.000000, T: 31648, Avg. loss: 0.041772
Total training time: 0.57 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.5s


Norm: 12.57, NNZs: 73, Bias: -2.000000, T: 31648, Avg. loss: 0.002465
Total training time: 0.53 seconds.
-- Epoch 1
Norm: 31.97, NNZs: 525, Bias: -2.000000, T: 31648, Avg. loss: 0.025404
Total training time: 0.53 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    1.1s remaining:    0.6s


Norm: 21.77, NNZs: 235, Bias: -3.000000, T: 31648, Avg. loss: 0.014029
Total training time: 0.52 seconds.
Norm: 55.14, NNZs: 1565, Bias: 4.000000, T: 31648, Avg. loss: 0.092770
Total training time: 0.59 seconds.


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.7s finished


Perceptron(max_iter=5, n_jobs=-1, verbose=10)

In [90]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-Food', 'B-Interior', 'B-Price', 'B-Service', 'B-Whole']

In [91]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

      B-Food       0.78      0.38      0.51       484
  B-Interior       0.83      0.60      0.70       182
     B-Price       0.83      0.45      0.59        33
   B-Service       0.83      0.60      0.70       359
     B-Whole       0.81      0.47      0.60       216

   micro avg       0.81      0.49      0.61      1274
   macro avg       0.82      0.50      0.62      1274
weighted avg       0.81      0.49      0.61      1274



In [92]:
sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, classes)

SGDClassifier()

In [93]:
print(classification_report(y_pred=sgd.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

      B-Food       0.75      0.44      0.56       484
  B-Interior       0.83      0.54      0.66       182
     B-Price       0.88      0.42      0.57        33
   B-Service       0.90      0.54      0.68       359
     B-Whole       0.75      0.62      0.68       216

   micro avg       0.81      0.52      0.63      1274
   macro avg       0.82      0.52      0.63      1274
weighted avg       0.81      0.52      0.63      1274



In [94]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01)

In [95]:
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels = new_classes))

              precision    recall  f1-score   support

      B-Food       0.33      0.83      0.47       484
  B-Interior       0.70      0.73      0.72       182
     B-Price       0.67      0.79      0.72        33
   B-Service       0.31      0.77      0.44       359
     B-Whole       0.55      0.87      0.67       216

   micro avg       0.38      0.81      0.52      1274
   macro avg       0.51      0.80      0.60      1274
weighted avg       0.42      0.81      0.54      1274



In [96]:
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier()

In [97]:
print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

      B-Food       0.78      0.31      0.44       484
  B-Interior       0.88      0.50      0.64       182
     B-Price       0.83      0.45      0.59        33
   B-Service       0.85      0.59      0.70       359
     B-Whole       0.75      0.62      0.68       216

   micro avg       0.81      0.47      0.60      1274
   macro avg       0.82      0.49      0.61      1274
weighted avg       0.81      0.47      0.59      1274



In [98]:
! pip install sklearn_crfsuite

In [99]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
            
getter = SentenceGetter(df)
sentences = getter.sentences